# CSCI 6390: Assignment 4
## Due October 14th
### By: Nicholas Lutrzykowski 
The goal of this assignment is to build a regression model to predict the number of shares of a news article. 


In [148]:
# Import Statements 
import numpy as np
import csv
import matplotlib.pyplot as plt
import warnings 
from sklearn.preprocessing import StandardScaler

### Import and setup data 

In [149]:
data_orig = np.genfromtxt('OnlineNewsPopularity.csv', delimiter=',')

data = np.concatenate((data_orig[1:,2:4], data_orig[1:, 7:13], data_orig[1:, 39:61]), axis=1)

num_points = data.shape[0]
num_attributes = data.shape[1]
print("Number of attributes:", num_attributes)
print("Number of points:", num_points)

# Read in the titles 
file = open('OnlineNewsPopularity.csv')
csvreader = csv.reader(file)
header = [] 
header = next(csvreader)
header = header[2:13] + header[39:]

Number of attributes: 30
Number of points: 39644


### Data Preprocessing

In [150]:
np.random.seed(42)
np.random.shuffle(data)
np.random.shuffle(data)
num_training, num_validation = 31715, 3964+31715


training = data[:num_training, :]
validation = data[num_training:num_validation, :]
test = data[num_validation:, :]
training_target, validation_target, test_target = training[:, -1], validation[:, -1], test[:, -1]

scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(training[:,:-1])
training = np.concatenate((np.ones((num_training,1), dtype=float),scaler.transform(training[:,:-1])), axis=1)

validation = np.concatenate((np.ones((num_validation-num_training,1), dtype=float), scaler.transform(validation[:,:-1])), axis=1)
test = np.concatenate((np.ones((num_points-num_validation,1), dtype=float), scaler.transform(test[:,:-1])), axis=1)

print("Training shape:", training.shape)
print("Validation shape:", validation.shape)
print("Test shape:", test.shape)


Training shape: (31715, 30)
Validation shape: (3964, 30)
Test shape: (3965, 30)


### Part I: Linear Regression via QR Factorization

In [151]:
def QR_Factorization(data): 
    Q = np.ones(data.shape, dtype=float)
    R = np.zeros((data.shape[1], data.shape[1]), dtype=float)
    R[0,0] = 1
    
    for i in range(1, Q.shape[1]):
        Q[:, i] = data[:, i]
        R[i, i] = 1
        for j in range(i): 
            pji = np.dot(data[:,i].T, Q[:,j])/np.dot(Q[:,j].T, Q[:,j])
            R[j, i] = pji 
            Q[:,i] -= np.dot(pji, Q[:,j])

    return Q, R
            

In [152]:
def back_sub(R, ortho_q_y):
    w = np.zeros((R.shape[0],))
    
    for i in range(R.shape[0]-1, -1, -1):
        w[i] = ortho_q_y[i]
        
        for j in range(i+1, R.shape[0]):
            w[i] -= w[j]*R[i,j]
        
    
    return w

In [153]:
def multiple_regression(training, Y):
    Q, R = QR_Factorization(training)

    ortho = np.matmul(Q.T, Q)
    ortho = 1/ortho
    ortho = np.where(ortho < 1e10, ortho, 0)
    ortho = np.where(ortho > -1e10, ortho, 0)
    
    ortho_q_y = np.matmul(np.matmul(ortho, Q.T), Y)
    
    w = back_sub(R, ortho_q_y)
    
    return w
    
    
    
    

In [154]:
def find_results(w, test, Y): 
    Y_hat = np.matmul(test, w)
    
    epsilon = Y-Y_hat
    SSE = np.dot(epsilon.T, epsilon)
    TSS = np.sum((Y-np.mean(Y))**2)
    
    R2 = (TSS-SSE)/TSS
    
    return SSE, TSS, R2
    

   

In [155]:
w = multiple_regression(training, training_target)

SSE, TSS, R2 = find_results(w, test, test_target)

print("SSE is:", SSE)
print("TSS is:", TSS)
print("R-squared is:", R2)



SSE is: 342677374158.571
TSS is: 350624971631.2273
R-squared is: 0.022666946497510915


Not sure why the above R-squared value is such a large negative number, I had a very small negative number prior to implementing part 2. I do not have time to debug why this is occuring. 

### Part II: Linear Regression with Regularization

In [156]:
rand_points = np.random.permutation(np.arange(training.shape[0]))
alphas = np.array([ 0, 1, 10, 100, 1000, 10000])
w_results = [] 
epsilon = 0.0001
eta = 1e-6

for alpha in alphas:
    dif = epsilon*2
    wprev = np.random.rand(num_attributes,)
  
    while dif > epsilon: 
        delta_w = -np.matmul(training.T, training_target) + np.matmul(np.matmul(training.T, training), wprev) + (alpha*wprev)
        w = wprev - (eta*delta_w)
        
        rand_points = np.random.permutation(rand_points)
        dif = np.sum((w - wprev)**2)
        wprev = w 
    
    w_results.append(wprev)



In [134]:
min_SSE = np.Inf
opt = np.Inf

for i in range(len(w_results)): 
    SSE, TSS, R2 = find_results(w_results[i], validation, validation_target)
    
    if SSE < min_SSE: 
        min_SSE = SSE
        opt = i

print("The best alpha value is:", alphas[opt])
print("The weights are:")
print(w_results[opt])

SSE, TSS, R2 = find_results(w_results[opt], test, test_target)

print("The R2 on the test data is:", R2)

The best alpha value is: 1000
The weights are:
[ 3.30292493e+03  1.08161584e+02  3.29107835e+01  4.35977519e+02
 -2.41736851e+02  1.63725109e+02  7.28187177e+01 -2.72636141e+02
  2.06026354e+02  8.28211705e+01 -1.69418621e+02 -4.44782174e+02
  5.03719550e+02 -2.48779334e+01  4.90667579e+02 -2.21722548e+01
 -1.40438608e+02  2.24494169e+01 -1.93411524e+02 -1.19207815e+02
  1.23813097e+00 -8.67214259e+01 -4.26607540e+01 -1.83463755e+02
  4.05839051e+01 -1.09235671e+02 -1.03649839e+01  4.34829415e+01
  5.96544530e+01  1.47237354e+02]
The R2 on the test data is: 0.024155528598761678
